# Intelligent Chat Bot for Korean Live Streaming Flatforms

Video live streaming services are interactive media contents between streamers and viewers by live chats. As streamers are willing to manage chats smarter, intelligent chat bots are needed. This project is making live stream chat bots with Gemma2 2B model as AfreecaTV(SOOP) extension program in order to detect questions from live chats and answer them automatically, or check inappropriate texts and remove various types of spams.

Primary goal is to classify questions from live chats and answer them by fine tuned Gemma model.

Project: Gemma Sprint Project @ 2024 Google Machine Learning Bootcamp Korea

Autor: Seonghyeok Jo, Kangwon National University.

In [12]:
!pip install -q websockets api scikit_learn==1.4.2 numpy pandas altair pyarrow==15.0.2 datasets peft trl accelerate transformers bitsandbytes wandb

## 1. Live Chat Crawling with AfreecaTV API


We need to get live chats for input. There is a magnificent code to crawl chats. Let's use this.

AfreecaTV Chat Crawler by **Soohyun-Chae(cha2hyun)** : [GitHub Repo](https://github.com/cha2hyun/afreecatv-chat-crawler)

In [13]:
import certifi
import ssl
import asyncio
import websockets
import requests

# 유니코드 및 기타 상수
# Unicode and other constants
F = "\x0c"
ESC = "\x1b\t"
SEPARATOR = "+" + "-" * 70 + "+"

In [14]:
# 아프리카TV에서 제공하는 API로 채팅 정보를 받습니다.
# Receives chat information using the API provided by AfreecaTV.
def get_player_live(bno, bid):
    url = 'https://live.afreecatv.com/afreeca/player_live_api.php'
    data = {
        'bid': bid,
        'bno': bno,
        'type': 'live',
        'confirm_adult': 'false',
        'player_type': 'html5',
        'mode': 'landing',
        'from_api': '0',
        'pwd': '',
        'stream_type': 'common',
        'quality': 'HD'
    }

    try:
        response = requests.post(f'{url}?bjid={bid}', data=data)
        # HTTP 요청 에러를 확인하고, 에러가 있을 경우 예외를 발생시킵니다.
        # Checks for HTTP request errors and raises an exception if there are any.
        response.raise_for_status()
        res = response.json()

        CHDOMAIN = res["CHANNEL"]["CHDOMAIN"].lower()
        CHATNO = res["CHANNEL"]["CHATNO"]
        FTK = res["CHANNEL"]["FTK"]
        TITLE = res["CHANNEL"]["TITLE"]
        BJID = res["CHANNEL"]["BJID"]
        CHPT = str(int(res["CHANNEL"]["CHPT"]) + 1)

        return CHDOMAIN, CHATNO, FTK, TITLE, BJID, CHPT

    except requests.RequestException as e:
        print(f"  ERROR: API 요청 중 오류 발생: {e}")
        return None
    except KeyError as e:
        print(f"  ERROR: 응답에서 필요한 데이터를 찾을 수 없습니다: {e}")
        return None

In [6]:
# SSL 컨텍스트 생성
# Create SSL context.
def create_ssl_context():
    ssl_context = ssl.create_default_context()
    ssl_context.load_verify_locations(certifi.where())
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
    return ssl_context

In [7]:
# 메시지 디코드 및 출력
# Decode messages and print.
def decode_message(bytes):
    parts = bytes.split(b'\x0c')
    messages = [part.decode('utf-8') for part in parts]
    if len(messages) > 5 and messages[1] not in ['-1', '1'] and '|' not in messages[1]:
        user_id, comment, user_nickname = messages[2], messages[1], messages[6]
        print(SEPARATOR)
        print(f"| {user_nickname}[{user_id}] - {comment}")
    else:
        # 채팅 뿐만 아니라 다른 메세지도 동시에 내려옵니다.
        # Not only chat messages, but other messages also come through at the same time.
        pass

In [8]:
# 바이트 크기 계산
# Calculate byte size.
def calculate_byte_size(string):
    return len(string.encode('utf-8')) + 6

In [9]:
# 채팅에 연결
# Connect to chat.
async def connect_to_chat(url, ssl_context):
    try:
        BNO, BID = url.split('/')[-1], url.split('/')[-2]
        CHDOMAIN, CHATNO, FTK, TITLE, BJID, CHPT = get_player_live(BNO, BID)
        print(f"{SEPARATOR}\n"
              f"  CHDOMAIN: {CHDOMAIN}\n  CHATNO: {CHATNO}\n  FTK: {FTK}\n"
              f"  TITLE: {TITLE}\n  BJID: {BJID}\n  CHPT: {CHPT}\n"
              f"{SEPARATOR}")
    except Exception as e:
        # API call failure.
        print(f"  ERROR: API 호출 실패 - {e}")
        return

    try:
        async with websockets.connect(
            f"wss://{CHDOMAIN}:{CHPT}/Websocket/{BID}",
            subprotocols=['chat'],
            ssl=ssl_context,
            ping_interval=None
        ) as websocket:
            # 최초 연결시 전달하는 패킷
            # Packet sent during the initial connection.
            CONNECT_PACKET = f'{ESC}000100000600{F*3}16{F}'
            # 메세지를 내려받기 위해 보내는 패킷
            # Packet sent to receive messages.
            JOIN_PACKET = f'{ESC}0002{calculate_byte_size(CHATNO):06}00{F}{CHATNO}{F*5}'
            # 주기적으로 핑을 보내서 메세지를 계속 수신하는 패킷
            # Packet that periodically sends pings to keep receiving messages.
            PING_PACKET = f'{ESC}000000000100{F}'

            await websocket.send(CONNECT_PACKET)
            print(f"  연결 성공, 채팅방 정보 수신 대기중...")
            await asyncio.sleep(2)
            await websocket.send(JOIN_PACKET)

            async def ping():
                while True:
                    # 5분동안 핑이 보내지지 않으면 소켓은 끊어집니다.
                    # Disconnects if ping is not recieved for 5 minutes.
                    await asyncio.sleep(60)  # 1분 = 60초
                    await websocket.send(PING_PACKET)

            async def receive_messages():
                while True:
                    data = await websocket.recv()
                    decode_message(data)

            await asyncio.gather(
                receive_messages(),
                ping(),
            )

    except Exception as e:
        # Web socket connection error.
        print(f"  ERROR: 웹소켓 연결 오류 - {e}")

This is the main function of crawling all the live chats from url's broadcast. The broadcast must be not private or age-restricted.

In [10]:
# Enter AfreecaTV URL.
# url = input("아프리카TV URL을 입력해주세요: ")
# ssl_context = create_ssl_context()
# await connect_to_chat(url, ssl_context)

# 2. Fine Tuning Gemma2-2b-it


There is a fine-tuned Gemma model for Korean.

Gemma2-2b-it model by **Google** : [Hugging Face](https://huggingface.co/google/gemma-2-2b-it)

In [15]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer
import torch

tokenizer = AutoTokenizer.from_pretrained(
    "google/gemma-2-2b-it",
)

model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# streamer = TextStreamer(tokenizer)

# messages = [
#     {"role": "user", "content": "대한민국의 수도에 대해 알려줘"},
# ]

# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device)

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|end_of_turn|>")
# ]

# outputs = model.generate(
#     input_ids,
#     max_new_tokens=512,
#     eos_token_id=terminators,
#     do_sample=False,
#     repetition_penalty=1.05,
#     streamer = streamer
# )
# response = outputs[0][input_ids.shape[-1]:]
# print(tokenizer.decode(response, skip_special_tokens=True))


Here is an example how to classify specific kinds of statement.

Question - Statement model by fine-tuned bert mini model by **Shahrukh Khan** :
[Hugging Face](https://huggingface.co/shahrukhx01/bert-mini-finetune-question-detection)

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("shahrukhx01/bert-mini-finetune-question-detection")
# model = AutoModelForSequenceClassification.from_pretrained("shahrukhx01/bert-mini-finetune-question-detection")

## 3-1. Calling Chat Dataset for Training


In [17]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [18]:
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from trl import SFTTrainer

train = pd.read_csv('/content/drive/MyDrive/datasets/train_chatbot.csv')

In [20]:
train["label"].value_counts(normalize=True)

,proportion
label,
0,0.761719
1,0.238281


In [21]:
import csv
import json
import json
import random
import torch
from datasets import Dataset, DatasetDict

# Read .csv
csv_file_path = '/content/drive/MyDrive/datasets/train_chatbot.csv'

with open(csv_file_path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Skip first line

    # Add to list after creating dictionary
    data = []
    for line in reader:
        d = {
            'input': line[0],
            'label': int(line[1])
        }
        data.append(d)

# json to string
json_string = json.dumps(data, ensure_ascii=False, indent=2)
json_file_path = '/content/drive/MyDrive/datasets/train_chatbot.json'

with open(json_file_path, 'w', encoding='utf-8') as f:
    f.write(json_string)

## 3-2. Training Model

In [3]:
import os
import copy
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    PreTrainedTokenizerBase,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType, PeftModel
import warnings

In [4]:
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTTrainer, SFTConfig
from accelerate import Accelerator

In [29]:
model_id = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    attn_implementation='eager'
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b-it/snapshots/299a8560bedf22ed1c72a8a11e7dce4a7f9f51f8/config.json
Model config Gemma2Config {
  "_name_or_path": "google/gemma-2-2b-it",
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": [
    1,
    107
  ],
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 2304,
  "initializer_range": 0.02,
  "intermediate_size": 9216,
  "max_position_embeddings": 8192,
  "model_type": "gemma2",
  "num_attention_heads": 8,
  "num_hidden_layers": 26,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "query_pre_attn_scalar": 256,
  "rms_norm_eps": 1e-06,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "torch_dtype": "bfloat16",


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Gemma2ForCausalLM.

All the weights of Gemma2ForCausalLM were initialized from the model checkpoint at google/gemma-2-2b-it.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Gemma2ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--google--gemma-2-2b-it/snapshots/299a8560bedf22ed1c72a8a11e7dce4a7f9f51f8/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": [
    1,
    107
  ],
  "pad_token_id": 0
}



In [22]:
data_path = json_file_path
train_data = Dataset.from_json(data_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [23]:
# Setting LoRA
lora_config = LoraConfig(
    r=6,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

In [24]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['input'])):
        prompt_list.append(f"""<bos><start_of_turn>user
{example['input'][i]}<end_of_turn>
<start_of_turn>model
{example['label'][i]}<end_of_turn><eos>""")
    return prompt_list

In [25]:
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
이거는<end_of_turn>
<start_of_turn>model
0<end_of_turn><eos>


In [30]:
# Setting Model
training_args = SFTConfig(
    output_dir = 'output',
    overwrite_output_dir = True,
    do_train = True,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    learning_rate = 5e-5,
    weight_decay = 0.01,
    num_train_epochs = 3,
    max_steps = -1,
    lr_scheduler_type = "cosine",
    warmup_ratio = 0.1,
    log_level = "info",
    logging_steps = 10,
    save_strategy = "epoch",
    bf16 = True,
    gradient_checkpointing = False,
    gradient_checkpointing_kwargs = {"use_reentrant": False},
    max_seq_length = 8,
    seed = 42,
    report_to = "none",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    args = training_args,
    formatting_func = generate_prompt
)

PyTorch: setting up devices


Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Using cpu_amp half precision backend


In [ ]:
model.config.use_cache = False

trainer.train()

***** Running training *****
  Num examples = 1,024
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3,072
  Number of trainable parameters = 2,614,341,888


In [ ]:
# 어댑터 모델 저장
ADAPTER_MODEL = "lora_adapter"
trainer.model.save_pretrained(ADAPTER_MODEL)

# 최종 모델 병합 및 저장
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16)

## 3-3. Load Model for Chat Filtering

In [ ]:
conversation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

## 3-4. Approved Chat Crawler Code with Filter

In [1]:
def filter_with_model(input_text):
    # Generate prompt
    messages = [{"role": "user", "content": input_text}]

    # Input to prompt format by tokenizer
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Response by Model
    response = conversation_pipeline(prompt, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, add_special_tokens=True)

    # Return response
    generated_text = response[0]["generated_text"]
    model_response = generated_text[len(prompt):]  # Delete input prompt
    return model_response

In [ ]:
# 아프리카TV에서 제공하는 API로 채팅 정보를 받습니다.
# Receives chat information using the API provided by AfreecaTV.
def get_player_live(bno, bid):
    url = 'https://live.afreecatv.com/afreeca/player_live_api.php'
    data = {
        'bid': bid,
        'bno': bno,
        'type': 'live',
        'confirm_adult': 'false',
        'player_type': 'html5',
        'mode': 'landing',
        'from_api': '0',
        'pwd': '',
        'stream_type': 'common',
        'quality': 'HD'
    }

    try:
        response = requests.post(f'{url}?bjid={bid}', data=data)
        # HTTP 요청 에러를 확인하고, 에러가 있을 경우 예외를 발생시킵니다.
        # Checks for HTTP request errors and raises an exception if there are any.
        response.raise_for_status()
        res = response.json()

        CHDOMAIN = res["CHANNEL"]["CHDOMAIN"].lower()
        CHATNO = res["CHANNEL"]["CHATNO"]
        FTK = res["CHANNEL"]["FTK"]
        TITLE = res["CHANNEL"]["TITLE"]
        BJID = res["CHANNEL"]["BJID"]
        CHPT = str(int(res["CHANNEL"]["CHPT"]) + 1)

        return CHDOMAIN, CHATNO, FTK, TITLE, BJID, CHPT

    except requests.RequestException as e:
        print(f"  ERROR: API 요청 중 오류 발생: {e}")
        return None
    except KeyError as e:
        print(f"  ERROR: 응답에서 필요한 데이터를 찾을 수 없습니다: {e}")
        return None

In [ ]:
# SSL 컨텍스트 생성
# Create SSL context.
def create_ssl_context():
    ssl_context = ssl.create_default_context()
    ssl_context.load_verify_locations(certifi.where())
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
    return ssl_context

In [ ]:
# 검열된 메시지 디코드 및 출력
# Decode filtered messages and print.
def decode_message_approved(bytes):
    parts = bytes.split(b'\x0c')
    messages = [part.decode('utf-8') for part in parts]
    if len(messages) > 5 and messages[1] not in ['-1', '1'] and '|' not in messages[1]:
        user_id, comment, user_nickname = messages[2], messages[1], messages[6]
        print(SEPARATOR)

        model_reply = filter_with_model(comment)  # Reply from model with comment.
        if(model_reply == 1):
            print(f"| {user_nickname}[{user_id}] - {comment}")

    else:
        # 채팅 뿐만 아니라 다른 메세지도 동시에 내려옵니다.
        # Not only chat messages, but other messages also come through at the same time.
        pass

In [ ]:
# 바이트 크기 계산
# Calculate byte size.
def calculate_byte_size(string):
    return len(string.encode('utf-8')) + 6

In [ ]:
# 채팅에 연결
# Connect to chat.
async def connect_to_chat_approved(url, ssl_context):
    try:
        BNO, BID = url.split('/')[-1], url.split('/')[-2]
        CHDOMAIN, CHATNO, FTK, TITLE, BJID, CHPT = get_player_live(BNO, BID)
        print(f"{SEPARATOR}\n"
              f"  CHDOMAIN: {CHDOMAIN}\n  CHATNO: {CHATNO}\n  FTK: {FTK}\n"
              f"  TITLE: {TITLE}\n  BJID: {BJID}\n  CHPT: {CHPT}\n"
              f"{SEPARATOR}")
    except Exception as e:
        # API call failure.
        print(f"  ERROR: API 호출 실패 - {e}")
        return

    try:
        async with websockets.connect(
            f"wss://{CHDOMAIN}:{CHPT}/Websocket/{BID}",
            subprotocols=['chat'],
            ssl=ssl_context,
            ping_interval=None
        ) as websocket:
            # 최초 연결시 전달하는 패킷
            # Packet sent during the initial connection.
            CONNECT_PACKET = f'{ESC}000100000600{F*3}16{F}'
            # 메세지를 내려받기 위해 보내는 패킷
            # Packet sent to receive messages.
            JOIN_PACKET = f'{ESC}0002{calculate_byte_size(CHATNO):06}00{F}{CHATNO}{F*5}'
            # 주기적으로 핑을 보내서 메세지를 계속 수신하는 패킷
            # Packet that periodically sends pings to keep receiving messages.
            PING_PACKET = f'{ESC}000000000100{F}'

            await websocket.send(CONNECT_PACKET)
            print(f"  연결 성공, 채팅방 정보 수신 대기중...")
            await asyncio.sleep(2)
            await websocket.send(JOIN_PACKET)

            async def ping():
                while True:
                    # 5분동안 핑이 보내지지 않으면 소켓은 끊어집니다.
                    # Disconnects if ping is not recieved for 5 minutes.
                    await asyncio.sleep(60)  # 1분 = 60초
                    await websocket.send(PING_PACKET)

            async def receive_messages_approved():
                while True:
                    data = await websocket.recv()
                    decode_message_approved(data)

            await asyncio.gather(
                receive_messages_approved(),
                ping(),
            )

    except Exception as e:
        # Web socket connection error.
        print(f"  ERROR: 웹소켓 연결 오류 - {e}")

# 4. Final Process

In [ ]:
# Enter AfreecaTV URL.
url = input("아프리카TV URL을 입력해주세요: ")
ssl_context = create_ssl_context()
await connect_to_chat_approved(url, ssl_context)